In [69]:
# Let's read in our document-term matrix
import pandas as pd
import pickle
from collections import defaultdict
# Import the necessary modules for LDA with gensim
# Terminal / Anaconda Navigator: conda install -c conda-forge gensim
from gensim import matutils, models, corpora
from gensim.models.phrases import Phrases, Phraser
import scipy.sparse
from sklearn.feature_extraction import text 
from sklearn.feature_extraction.text import CountVectorizer
from nltk import word_tokenize, pos_tag
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
import spacy
nlp = spacy.load('en_core_web_sm')

dtm = pd.read_pickle('data_files/comedian_dtm')
raw_data = pd.read_pickle('data_files/comedian_corpus')
my_stop_words = pd.read_pickle('data_files/stop_words')

In [109]:
from pyLDAvis import gensim_models
import warnings
warnings.filterwarnings('ignore')

C:\Users\mattp\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
dtm

,Bo Burnham - Words Words Words,Bo Burnham - What,Bo Burnham - Make Happy,John Mulaney - New in Town,John Mulaney - Comeback Kid,John Mulaney - Kid Gorgeous,Ricky Gervais - Politics,Ricky Gervais - Science,Ricky Gervais - Humanity,Kevin Bridges - Story So Far,Kevin Bridges - The Story Continues,Kevin Bridges - Whole Different Story,Aziz Ansari - Intimate Moments Sensual Evening,Aziz Ansari - Madison Square Garden,Aziz Ansari - Right Now
aaaaahhhhhhh,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
aaaaauuugghhhhhh,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
aaaahhhhh,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
aaaand,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
aaand,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zoologist,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
zs,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
zuckerberg,0,0,0,0,0,0,0,0,0,0,2,1,0,0,0
zuki,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [5]:
dtm.loc['yeahhhhhhhhhhhhh']

Bo Burnham - Words Words Words                    0
Bo Burnham - What                                 0
Bo Burnham - Make Happy                           0
John Mulaney - New in Town                        1
John Mulaney - Comeback Kid                       0
John Mulaney - Kid Gorgeous                       0
Ricky Gervais - Politics                          0
Ricky Gervais - Science                           0
Ricky Gervais - Humanity                          0
Kevin Bridges - Story So Far                      0
Kevin Bridges - The Story Continues               0
Kevin Bridges - Whole Different Story             0
Aziz Ansari - Intimate Moments Sensual Evening    0
Aziz Ansari - Madison Square Garden               0
Aziz Ansari - Right Now                           0
Name: yeahhhhhhhhhhhhh, dtype: int64

# Attempt 1

In [6]:
# We're going to put the term-document matrix into a new gensim format, from df --> sparse matrix --> gensim corpus
sparse_counts = scipy.sparse.csr_matrix(dtm)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [7]:
cv = CountVectorizer(stop_words = my_stop_words)
data_cv = cv.fit(raw_data.sketch_text)
id2word = dict((v, k) for k, v in data_cv.vocabulary_.items())

In [8]:
# Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term),
# we need to specify two other parameters as well - the number of topics and the number of passes
lda = models.LdaModel(corpus=corpus, id2word = id2word, num_topics=2, passes=20)
lda.print_topics()

[(0,
  '0.007*"fucking" + 0.006*"going" + 0.004*"want" + 0.004*"good" + 0.004*"old" + 0.004*"went" + 0.004*"fuck" + 0.004*"day" + 0.003*"come" + 0.003*"years"'),
 (1,
  '0.006*"okay" + 0.006*"went" + 0.006*"god" + 0.005*"going" + 0.004*"want" + 0.004*"fucking" + 0.004*"sort" + 0.003*"thank" + 0.003*"funny" + 0.003*"come"')]

In [9]:
lda = models.LdaModel(corpus=corpus, id2word = id2word, num_topics=4, passes=20)
lda.print_topics()

[(0,
  '0.010*"fucking" + 0.007*"going" + 0.006*"fuckin" + 0.005*"good" + 0.004*"mate" + 0.004*"come" + 0.004*"wee" + 0.004*"shite" + 0.004*"need" + 0.004*"kind"'),
 (1,
  '0.007*"shit" + 0.006*"randy" + 0.006*"hey" + 0.006*"whoa" + 0.006*"goes" + 0.005*"fuck" + 0.005*"thank" + 0.004*"come" + 0.004*"harris" + 0.004*"cool"'),
 (2,
  '0.008*"okay" + 0.006*"going" + 0.005*"want" + 0.005*"uh" + 0.004*"old" + 0.004*"hey" + 0.004*"god" + 0.004*"love" + 0.004*"good" + 0.004*"look"'),
 (3,
  '0.008*"fucking" + 0.007*"went" + 0.006*"going" + 0.005*"funny" + 0.005*"ill" + 0.004*"want" + 0.004*"mate" + 0.003*"day" + 0.003*"big" + 0.003*"come"')]

In [10]:
lda = models.LdaModel(corpus=corpus, id2word = id2word, num_topics=6, passes=20)
lda.print_topics()

[(0,
  '0.009*"uh" + 0.008*"fucking" + 0.008*"going" + 0.005*"okay" + 0.004*"mate" + 0.004*"shite" + 0.004*"shit" + 0.004*"stuff" + 0.004*"fuck" + 0.004*"trying"'),
 (1,
  '0.007*"went" + 0.005*"funny" + 0.005*"ill" + 0.005*"want" + 0.004*"day" + 0.004*"old" + 0.004*"make" + 0.004*"going" + 0.004*"look" + 0.004*"good"'),
 (2,
  '0.008*"going" + 0.007*"fucking" + 0.005*"old" + 0.005*"good" + 0.004*"day" + 0.004*"fuckin" + 0.004*"look" + 0.004*"come" + 0.003*"dad" + 0.003*"years"'),
 (3,
  '0.009*"fucking" + 0.006*"love" + 0.005*"went" + 0.005*"want" + 0.005*"stuff" + 0.005*"joke" + 0.005*"didnt" + 0.004*"bo" + 0.004*"going" + 0.004*"old"'),
 (4,
  '0.000*"going" + 0.000*"fucking" + 0.000*"okay" + 0.000*"went" + 0.000*"come" + 0.000*"good" + 0.000*"want" + 0.000*"hey" + 0.000*"ill" + 0.000*"fuck"'),
 (5,
  '0.009*"okay" + 0.006*"god" + 0.006*"going" + 0.006*"want" + 0.006*"hey" + 0.006*"fuck" + 0.005*"fucking" + 0.005*"shit" + 0.005*"thank" + 0.004*"went"')]

# Attempt 2 - Nouns Only

In [11]:
# Let's create a function to pull out nouns from a string of text
def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)

In [12]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns = pd.DataFrame(raw_data.sketch_text.apply(nouns))
data_nouns

,sketch_text
Bo Burnham - Words Words Words,thank i hey hey ho thats hitler power show bit...
Bo Burnham - What,macdonald farm e i o farm pig e i i snort macd...
Bo Burnham - Make Happy,thing mother person life i mother person uh uh...
John Mulaney - New in Town,town john town hes mustard shirt hes papers he...
John Mulaney - Comeback Kid,charm wit snl writer john mulaney marriage bee...
John Mulaney - Kid Gorgeous,welcome city music time questions walk hi im j...
Ricky Gervais - Politics,houses parliament home politics politics polit...
Ricky Gervais - Science,voice laboratory sort frankensteins doctors bo...
Ricky Gervais - Humanity,hello thank fuck thank im gon youre weve money...
Kevin Bridges - Story So Far,im crossroads lines moon time walking devil i ...


In [104]:
cvn = CountVectorizer(stop_words = my_stop_words)
data_dict_cvn = cvn.fit(data_nouns.sketch_text)
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())
data_dtm_cvn = cvn.fit_transform(data_nouns.sketch_text)
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names())
data_dtmn.index = data_nouns.index
dtmn = data_dtmn.transpose()

C:\Users\mattp\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NameError: name 'data_cvn' is not defined

In [ ]:
sparse_counts = scipy.sparse.csr_matrix(dtmn)
corpusn = matutils.Sparse2Corpus(sparse_counts)

In [ ]:
ldan = models.LdaModel(corpus=corpusn, id2word = id2wordn, num_topics=2, passes=20)
ldan.print_topics()

In [ ]:
ldan = models.LdaModel(corpus=corpusn, id2word = id2wordn, num_topics=4, passes=20)
ldan.print_topics()

In [ ]:
ldan = models.LdaModel(corpus=corpusn, id2word = id2wordn, num_topics=6, passes=20)
ldan.print_topics()

Neither of these options are currently giving coherent topics, with two main issues:
- The topic is picking up a word that isn't mentioned that often such as 'yeahhhhhhh'
- The majority of the words in our topic are still too generic 

To solve these problems, I will:
- Remove any words from my corpus that only appear once, as they add minimal input to my topics but would take a lot of time to clean
- Look to lemmatize my raw texts before creating my document term matrix

In [48]:
pattern = "[\w'.]+|[£|$0-9]+"
tokenizer = RegexpTokenizer(pattern)
def tokenize_text(text):
    return tokenizer.tokenize(text) 

In [59]:
lemmatizer = WordNetLemmatizer()
def lemmatize(list_of_text):
    text = [lemmatizer.lemmatize(word) for word in list_of_text]
    return text

In [84]:
def remove_stops(list_of_text):
    text = [word for word in list_of_text if word not in my_stop_words]
    return text

In [78]:
def nlp_pipeline(text):
    text = tokenize_text(text)
    text = remove_stops(text)
    text = lemmatize(text)
    text = ' '.join(text)
    return text

In [79]:
def unigrams(df, col_name, min_number):
    tokenized_docs = df[col_name].apply(lambda x: tokenize_text(x))
    frequency = defaultdict(int)
    for doc in tokenized_docs:
        for token in doc:
            frequency[token] +=1
    processed_tokens = [[token for token in doc if frequency[token] > min_number] for doc in tokenized_docs]
    return processed_tokens

In [64]:
def bigrams(df, col_name, min_number_b):
    tokenized_docs = df[col_name].apply(lambda x: tokenize_text(x))
    input_tokens = [doc for doc in tokenized_docs]
    bigram_model = Phraser(Phrases(input_tokens, min_count = min_number_b, threshold = min_number_b, delimiter = b' '))
    bigram_tokens = []
    for doc_tokens in input_tokens:
        bigram_tokens.append(bigram_model[doc_tokens])
    return bigram_model, bigram_tokens

In [35]:
def ngrams(df, col_name, min_number, min_number_b):
    # Create dict and bow cased on unigrams only
    processed_unigrams = unigrams(df, col_name, min_number)
    dictionary = corpora.Dictionary(processed_unigrams)
    bow_corpus = [dictionary.doc2bow(text) for text in processed_unigrams]
    
    # Add in bigrams to the model
    bigram_model, bigram_tokens = bigrams(df, col_name, min_number_b)
    dictionary.add_documents(bigram_tokens)
    bow_corpus = [dictionary.doc2bow(text) for text in bigram_model[processed_unigrams]]
    return dictionary, bow_corpus

In [105]:
def lda_model(corpus, dictionary, num_topics, random_state  = 100):
    lda_model = models.LdaMulticore(corpus = corpus, id2word = dictionary, num_topics = num_topics, random_state = random_state, passes = 20)
    return lda_model

C:\Users\mattp\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [110]:
def print_topic_keywords(lda_model, dictionary, num_topics):
    topics = []
    for n in range(num_topics):
        topics.append([])
        
    for topic_count in range(0, num_topics):
        for topic_term in lda_model.get_topic_terms(topic_count):
            topics[topic_count].append(dictionary.id2token[topic_term[0]])
            
    for counter, topic in enumerate(topics):
        print(f'Topic {counter}: {topic}')

In [111]:
def model_prep(df, col_name, min_number, min_number_b):
    df['prepped_text'] = df[col_name].apply(lambda x: nlp_pipeline(x))
    dictionary, bow_corpus = ngrams(df, 'prepped_text', min_number, min_number_b)
    return dictionary, bow_corpus
lda_dict, lda_corpus = model_prep(raw_data, 'sketch_text', 1, 2)

In [112]:
num_topics = 2
lda_model_output = lda_model(corpus = lda_corpus, dictionary = lda_dict, num_topics = num_topics)
print_topic_keywords(lda_model_output, lda_dict, num_topics)

Topic 0: ['going', 'day', 'kid', 'mean', 'look', 'fucking', 'come', 'went', 'big', 'good']
Topic 1: ['fucking', 'going', 'want', 'okay', 'come', 'went', 'uh', 'fuck', 'day', 'look']


In [113]:
num_topics = 4
lda_model_output = lda_model(corpus = lda_corpus, dictionary = lda_dict, num_topics = num_topics)
print_topic_keywords(lda_model_output, lda_dict, num_topics)

Topic 0: ['okay', 'going', 'went', 'fucking', 'god', 'mean', 'sort', 'day', 'um', 'look']
Topic 1: ['fucking', 'going', 'went', 'come', 'mate', 'fuck', 'day', 'good', 'want', 'ill']
Topic 2: ['okay', 'uh', 'want', 'love', 'look', 'day', 'good', 'there', 'going', 'let']
Topic 3: ['fucking', 'went', 'day', 'didnt', 'joke', 'there', 'nut', 'going', 'tweet', 'big']


In [114]:
num_topics = 6
lda_model_output = lda_model(corpus = lda_corpus, dictionary = lda_dict, num_topics = num_topics)
print_topic_keywords(lda_model_output, lda_dict, num_topics)

Topic 0: ['bill clinton', 'kid', 'day', 'mom', 'dad', 'hey', 'wife', 'baby', 'cow', 'look']
Topic 1: ['okay', 'went', 'going', 'fucking', 'um', 'come', 'god', 'fuck', 'sort', 'want']
Topic 2: ['okay', 'uh', 'want', 'good', 'love', 'kind', 'there', 'woman', 'day', 'look']
Topic 3: ['look', 'kid', 'doctor', 'girlfriend', 'day', 'lot', 'hey', 'didnt', 'drinking', 'mean']
Topic 4: ['want', 'going', 'went', 'day', 'look', 'come', 'love', 'okay', 'good', 'saw']
Topic 5: ['fucking', 'going', 'mate', 'fuckin', 'fuck', 'big', 'day', 'went', 'come', 'ill']


These topics look a lot better than our previous attempts, but there is still some overlap in them. To further investigate them, we can look at the more distinct terms in each topic using pyLDAvis.

In [115]:
pyLDAvis.enable_notebook()
vis = gensim_models.prepare(lda_model_output, lda_corpus, dictionary = lda_model_output.id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.045615  0.023701       1        1  26.365784
4      0.014306  0.022786       2        1  22.505208
5      0.079886 -0.099478       3        1  22.378311
1      0.050868  0.075583       4        1  15.862399
3     -0.085775  0.017006       5        1   6.479824
0     -0.104901 -0.039598       6        1   6.408474, topic_info=              Term        Freq       Total Category  logprob  loglift
1296       fucking  247.000000  247.000000  Default  30.0000  30.0000
3493        fuckin   54.000000   54.000000  Default  29.0000  29.0000
521            kid  119.000000  119.000000  Default  28.0000  28.0000
5756  bill clinton   19.000000   19.000000  Default  27.0000  27.0000
635            mom   34.000000   34.000000  Default  26.0000  26.0000
...            ...         ...         ...      ...      ...      ...
370         friend    6.484943   83.371829   Topic6  -6.0295   0.1937
398          going    6.485787  215.025207   Topic6  -6.0294  -0.7536
401           good    6.485705  148.132534   Topic6  -6.0294  -0.3810
661           need    6.485653  104.889068   Topic6  -6.0294  -0.0358
552            let    6.485498  110.973823   Topic6  -6.0294  -0.0922

[467 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
3188      4  0.824849       affect
11        1  0.241996         aint
11        3  0.080665         aint
11        4  0.645323         aint
5733      6  0.741787  alpha house
...     ...       ...          ...
1123      2  0.203706        young
1123      3  0.116404        young
1123      4  0.029101        young
1123      5  0.058202        young
1123      6  0.029101        young

[918 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 5, 6, 2, 4, 1])

In [116]:
all_topics = {}
lambd = 0.5
for i in range(1,num_topics+1):
    topic = vis.topic_info[vis.topic_info.Category == 'Topic'+str(i)]
    topic['relevance'] = topic['loglift']*(1-lambd) + topic['logprob']*lambd
    all_topics['Topic '+str(i)] = topic.sort_values(by = 'relevance', ascending = False).Term[:20].values

relevant_terms = pd.DataFrame(all_topics)

In [103]:
relevant_terms

C:\Users\mattp\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6
0,going,went,fucking,fucking,woman,come
1,fucking,fucking,going,went,going,day
2,good,fuck,come,day,day,kid
3,look,love,there,going,fucking,fucking
4,want,day,went,kid,there,went
5,let,want,ill,didnt,love,okay
6,mean,look,okay,good,come,need
7,okay,okay,kid,there,god,want
8,woman,going,want,look,shit,big
9,fuck,come,start,want,okay,let
